# Нелинейные методы снижения размерности

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### PCA с ядрами

Так как PCA фактически работает не исходными признаками, а с матрицей их ковариаций, можно
использовать для ее вычисления вместо скалярного произведения $\langle x_i, x_j \rangle$ произвольное
ядро $K(x_i, x_j)$. Это будет соответствовать переходу в другое пространство, в котором
наше предположение о линейности уже будет иметь смысл. Единственная проблема - непонятно, как
подбирать ядро.

Ниже приведены примеры объектов в исходном пространстве (похожие группы обозначены одним цветом
для наглядности), и результат их трансформации в новые пространства (для разных ядер). Если результаты
получаются линейно разделимыми - значит мы выбрали подходящее ядро.

In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.datasets import make_circles, make_moons, make_blobs

def KPCA_show(X, y):
    reds = y == 0
    blues = y == 1
    
    plt.figure(figsize=(8, 8))
    rows, cols = 2, 2
    plt.subplot(rows, cols, 1)
    plt.scatter(X[reds, 0], X[reds, 1], alpha=0.5, c='r')
    plt.scatter(X[blues, 0], X[blues, 1], alpha=0.5, c='b')
    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    
    kernels_params = [
        dict(kernel='rbf', gamma=10),
        dict(kernel='poly', gamma=10),
        dict(kernel='cosine', gamma=10),
    ]
    
    for i, p in enumerate(kernels_params):
        dec = KernelPCA(**p)
        X_transformed = dec.fit_transform(X)
        
        plt.subplot(rows, cols, i + 2)
        plt.scatter(X_transformed[reds, 0], X_transformed[reds, 1], alpha=0.5, c='r')
        plt.scatter(X_transformed[blues, 0], X_transformed[blues, 1], alpha=0.5, c='b')
        ax = plt.gca()
        ax.set_aspect('equal', adjustable='box')
        
np.random.seed(54242)
KPCA_show(*make_circles(n_samples=1000, factor=0.2, noise=0.1))

In [ ]:
np.random.seed(54242)
KPCA_show(*make_moons(n_samples=1000, noise=0.1))

# t-SNE

Возьмем датасет MNIST и сделаем проекции в двумерное пространство с помощью PCA, LDA и t-SNE.

In [ ]:
# Authors: Fabian Pedregosa <fabian.pedregosa@inria.fr>
#          Olivier Grisel <olivier.grisel@ensta.org>
#          Mathieu Blondel <mathieu@mblondel.org>
#          Gael Varoquaux
# License: BSD 3 clause (C) INRIA 2011

print(__doc__)
from time import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30


#----------------------------------------------------------------------
# Scale and visualize the embedding vectors
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(digits.data.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)


#----------------------------------------------------------------------
# Plot images of the digits
n_img_per_row = 20
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img[ix:ix + 8, iy:iy + 8] = X[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title('A selection from the 64-dimensional digits dataset')


#----------------------------------------------------------------------
# Projection on to the first 2 principal components

print("Computing PCA projection")
t0 = time()
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(X)
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0))

#----------------------------------------------------------------------
# Projection on to the first 2 linear discriminant components

print("Computing Linear Discriminant Analysis projection")
X2 = X.copy()
X2.flat[::X.shape[1] + 1] += 0.01  # Make X invertible
t0 = time()
X_lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components=2).fit_transform(X2, y)
plot_embedding(X_lda,
               "Linear Discriminant projection of the digits (time %.2fs)" %
               (time() - t0))

#----------------------------------------------------------------------
# t-SNE embedding of the digits dataset
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(X)

plot_embedding(X_tsne,
               "t-SNE embedding of the digits (time %.2fs)" %
               (time() - t0))

plt.show()

Кроме проекции в двумерное пространство мы, очевидно, получили кластеризацию чисел. То есть одним из приложений этих методов является кластеризация.

На прошлом семинаре мы сравнивали различные способы *линейного* снижения размерности на примере датасета "ирисы". Сегодня мы сравним результаты PCA (линейный способ снижения размерности) с ядровой разновидностью PCA и с t-SNE.

# Внимание, конкурс! 

Сейчас вам предстоит поэкспериментировать с различными нелинейными методами снижения размерности в задаче "ирисы". Победит тот, кто получит наилучшее качество заданного классификатора после снижения размерности. Если несколько человек получат одинаковое наилучшее качество, то победителем считается тот, кто показал свое решение первым.

Время на выполнение задания: 30 минут.

In [ ]:
import seaborn as sns; sns.set(style='white')
from sklearn import datasets
from mpl_toolkits.mplot3d import Axes3D

# Загрузим наши ирисы
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Заведём красивую трёхмерную картинку
fig = plt.figure(1, figsize=(6, 5))
plt.clf()
ax = Axes3D(fig, rect=[0, 0, .95, 1], elev=48, azim=134)

plt.cla()

for name, label in [('Setosa', 0), ('Versicolour', 1), ('Virginica', 2)]:
    ax.text3D(X[y == label, 0].mean(),
              X[y == label, 1].mean() + 1.5,
              X[y == label, 2].mean(), name,
              horizontalalignment='center',
              bbox=dict(alpha=.5, edgecolor='w', facecolor='w'))
# Поменяем порядок цветов меток, чтобы они соответствовали правильному
y_clr = np.choose(y, [1, 2, 0]).astype(np.float)
ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y_clr, cmap=plt.cm.spectral)

ax.w_xaxis.set_ticklabels([])
ax.w_yaxis.set_ticklabels([])
ax.w_zaxis.set_ticklabels([])

In [ ]:
from sklearn.decomposition import PCA

# Прогоним встроенный в sklearn PCA
pca = PCA(n_components=2)
X_centered = X - X.mean(axis=0)
pca.fit(X_centered)
X_pca = pca.transform(X_centered)

def plot_projection(X,y):
    plt.plot(X[y == 0, 0], X[y == 0, 1], 'bo', label='Setosa')
    plt.plot(X[y == 1, 0], X[y == 1, 1], 'go', label='Versicolour')
    plt.plot(X[y == 2, 0], X[y == 2, 1], 'ro', label='Virginica')
    plt.legend(loc=0);
    plt.show()
    
plot_projection(X_pca,y)

Посмотрим на качество классификатора DecisionTree на исходных признаках.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

def fit_predict(X, y):

    # Повторим то же самое разбиение на валидацию и тренировочную выборку.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, 
                                                        stratify=y, 
                                                        random_state=42)

    clf = DecisionTreeClassifier(max_depth=2, random_state=42)
    clf.fit(X_train, y_train)
    preds = clf.predict_proba(X_test)
    print('Accuracy: {:.5f}'.format(accuracy_score(y_test, 
                                                    preds.argmax(axis=1))))
    
fit_predict(X_pca,y)

Теперь ваша очередь пробовать новые методы. Чем больше методов вы попробуете, тем больше шанс, что именно вы получите лучшее качество!

Не забудьте попробовать следующие методы:

a) t-SNE

b) PCA с ядром

c) Можете использовать LDA (линейный метод из прошлого семинара), хотя это не обязательно

Код-заготовка для использования t-SNE

In [ ]:
from sklearn import manifold

#init: 'random' or 'pca'
#learning_rate может быть любым. Попробуйте такие learning_rate: 1, 10, 100, 0.1, 0.01, ... Экспериментируйте!

tsne = manifold.TSNE(n_components=2, init=..., learning_rate=...)
X_tsne = ...

plot_projection(X_tsne,y)
fit_predict(X_tsne,y)

Код-заготовка для использования Kernel PCA

In [ ]:
#виды ядер (kernel) вы можете найти в этом ноутбуке выше, gamma может быть любым числом. Подбирайте параметры!
kpca = KernelPCA(n_components=2, kernel=..., gamma=...)
X_kpca = ...

#Не забудьте визуализировать новые признаки и вывести на экран качество алгоритма на этих признаках.

**Выводы.** Напишите, какой метод снижения размерности, на ваш взгляд, работает лучше всего в данной задаче? И почему так может происходить?

Какое наилучшее качество вам удалось получить?